In [1]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

In [2]:
api_key = 'affdac5f248f69b65c935406621fe7ca9e349a23'
years = list(range(2010,2024))

all_data = []

for year in years:
    base_url = f"https://api.census.gov/data/{year}/acs/acs5/profile"
    params = {
        "get": "NAME,DP03_0086E,DP03_0009PE,DP03_0088E,DP03_0119PE,DP03_0128PE",
        "for": "state:*"
    }
    response = requests.get(base_url, params = params)
    if response.status_code == 200:
        data = response.json()
        columns = data[0]
        rows = data[1:]
        df = pd.DataFrame(rows,columns = columns)
        df['Year'] = year
        all_data.append(df)

if all_data:
    final_df = pd.concat(all_data, ignore_index = True)
    final_df = final_df[['Year', 'NAME', 'DP03_0086E', 'DP03_0009PE', 'DP03_0088E', 'DP03_0119PE', 'DP03_0128PE']]
    final_df.columns = ['Year', 'State', 'Median Family Income', 'Unemployment Rate', 'Per Capita Income', 'Perc Families BPL', 'Perc Population BPL']

for col in ['Median Family Income', 'Unemployment Rate', 'Per Capita Income', 'Perc Families BPL', 'Perc Population BPL']:
    final_df[col] = pd.to_numeric(final_df[col], errors = 'coerce')

final_df = final_df[~(final_df['State']=='Puerto Rico')]
final_df.sort_values(by = ['State','Year'], inplace = True, ignore_index = True)

final_df.to_excel('economic_characteristics_2010_2023.xlsx', sheet_name = 'Economic Data')
